# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_data = pd.read_csv("../output_data/cities-Rilee.csv")
weather_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,norman wells,65.2820,-126.8329,3.24,85,40,2.30,CA,1677455937
1,1,hambantota,6.1241,81.1185,72.14,72,99,9.08,LK,1677455937
2,2,avarua,-21.2078,-159.7750,84.25,65,20,9.22,CK,1677455938
3,3,hobart,-42.8794,147.3294,61.05,73,75,13.80,AU,1677455654
4,4,bluff,-46.6000,168.3333,63.81,74,67,7.58,NZ,1677455938
...,...,...,...,...,...,...,...,...,...,...
547,547,montepuez,-13.1256,38.9997,68.09,99,100,3.02,MZ,1677456147
548,548,kempele,64.9131,25.5034,14.02,90,100,5.99,FI,1677456147
549,549,pingliang,35.5392,106.6861,34.77,90,13,0.34,CN,1677456148
550,550,galle,6.0367,80.2170,72.77,87,100,5.32,LK,1677456148


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configuring gmaps
gmaps.configure(api_key=g_key)

#assinging latitude and Longitude to their variable
locations = weather_data[["Lat", "Lng"]]

#assigning Humidity to it's variable
humidity = weather_data["Humidity"]

In [5]:
#Adding Heatmap Layer to the Map
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

#Creating the Heat Layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

#Adding the layer
fig.add_layer(heat_layer)

#Display the Heat Map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#create new list only including cities with wind speed less than 10mph, cloudiness equals 0, and max temp between 71 and 79
refined_city_df = weather_data.loc[(weather_data["Wind Speed"] <10) & (weather_data["Cloudiness"] == 0) & (weather_data["Max Temp"] >= 71) & (weather_data["Max Temp"] <= 79)].dropna()
refined_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,pinar del rio,22.4175,-83.6981,72.21,57,0,4.25,CU,1677455941
74,74,saint-pierre,-21.3393,55.4781,74.91,73,0,4.61,RE,1677455960
260,260,chauk,20.8833,94.8167,72.86,37,0,4.92,MM,1677456037
267,267,santa maria,-29.6842,-53.8069,76.30,78,0,5.75,BR,1677456041
305,305,sur,22.5667,59.5289,74.62,54,0,5.28,OM,1677456054
380,380,saint pete beach,27.7253,-82.7412,72.18,79,0,4.61,US,1677456086
444,444,moron,-34.6534,-58.6198,73.98,60,0,8.81,AR,1677456110
470,470,conceicao da barra,-18.5933,-39.7322,78.03,84,0,7.87,BR,1677456119
532,532,puri,19.8000,85.8500,73.92,74,0,4.05,IN,1677456140


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Create the Hotel DataFrame
hotel_df = refined_city_df.loc[:,["City", "Country", "Lat", "Lng"]]

#Add a Hotel Name column to the Hotel DataFrame
hotel_df["Hotel Name"] = ""

#Display entire Hotel DataFrame
hotel_df

,City,Country,Lat,Lng,Hotel Name
12,pinar del rio,CU,22.4175,-83.6981,
74,saint-pierre,RE,-21.3393,55.4781,
260,chauk,MM,20.8833,94.8167,
267,santa maria,BR,-29.6842,-53.8069,
305,sur,OM,22.5667,59.5289,
380,saint pete beach,US,27.7253,-82.7412,
444,moron,AR,-34.6534,-58.6198,
470,conceicao da barra,BR,-18.5933,-39.7322,
532,puri,IN,19.8000,85.8500,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
         "keyword" : "hotel", 
         "radius" : 5000,
         "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 12: pinar del rio.
Missing field/result... skipping.
------------
Retrieving Results for Index 74: saint-pierre.
Missing field/result... skipping.
------------
Retrieving Results for Index 260: chauk.
Missing field/result... skipping.
------------
Retrieving Results for Index 267: santa maria.
Missing field/result... skipping.
------------
Retrieving Results for Index 305: sur.
Missing field/result... skipping.
------------
Retrieving Results for Index 380: saint pete beach.
Missing field/result... skipping.
------------
Retrieving Results for Index 444: moron.
Missing field/result... skipping.
------------
Retrieving Results for Index 470: conceicao da barra.
Missing field/result... skipping.
------------
Retrieving Results for Index 532: puri.
Missing field/result... skipping.
------------
-------End of Search-------


In [12]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
12,pinar del rio,CU,22.4175,-83.6981,
74,saint-pierre,RE,-21.3393,55.4781,
260,chauk,MM,20.8833,94.8167,
267,santa maria,BR,-29.6842,-53.8069,
305,sur,OM,22.5667,59.5289,
380,saint pete beach,US,27.7253,-82.7412,
444,moron,AR,-34.6534,-58.6198,
470,conceicao da barra,BR,-18.5933,-39.7322,
532,puri,IN,19.8000,85.8500,


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))